## 맛있는 절약 : 평가용 dataset 생성
1. find_keyIngredients_tasty : (사전 생성) 레시피의 핵심 맛과 핵심 재료를 판단
2. generate_food_group_radio : (사전 생성) 레시피의 식품군을 판별
3. fridge_recipe_transform : (기능) 냉장고 재료 대체 
4. simple_recipe_transform : (기능) 레시피 간소화 
5. balance_nutrition : (기능) 영양 맞춤형 레시피 

## 일반적인 평가용 dataset 생성 case
- 검색한 것이 질문과 관련이 있는가?
- 답변이 질문과 관련이 있는가?
- 답변을 검색한 문서 안에서 답변한 것인가?(할루시네이션 check)

## 한계
검색한 결과에 대한 정확한 기준 데이터(Ground truth) 구축이 사실상 어려움
따라서, 검색한 결과에 대한 Ground truth가 존재한다면 모두 데이터셋으로 활용하고, 아니라면 질문과 답변만으로 데이터셋을 구축하여 활용함
⇒ 우리는 레시피 생성시에 RAG가 없기 때문에 질문과 답변만으로 데이터셋을 구축해서 활용함

In [4]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [5]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("맛있는 절약-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
맛있는 절약-Evaluations


## 레시피 dataset
spicy_level, type_key, method_key, servings를 기준으로 각각 중복되지 않은 행만 추출 

In [13]:
import pandas as pd
df_recipeinfo = pd.read_csv("data/diverse_recipes_dataset.csv")
df_recipeinfo.describe()

,_id,title,type_key,method_key,servings,cooking_time,difficulty,ingredients,cooking_steps,tips,recipe_type,spicy_level
count,37,37,37,37,37,33,35,37,37,37,37,37
unique,37,37,12,13,6,7,2,37,37,22,37,6
top,673c36d38fad4e3c931171ba,"토마토 마리네이드&냉파스타, 새콤달콤 시원한 별미",반찬,볶음,2인분,30분 이내,초급,"['방울토마토(1.5kg)', '양파(1개)', '파슬리가루(1스푼)', '올리브오...","['양파는 잘게 다져 준비해주세요양파(소) 1개', '방울토마토는 끓는물에 약 2분...",[],"['채식', '저칼로리', '새콤달콤한맛', '양식', '가벼운']",매운맛없음
freq,1,1,12,8,12,11,34,1,1,16,1,25


In [14]:
df_userinfo = pd.read_csv('data/userinfo_example_dataset.csv')
df_userinfo.describe()


,user_allergy_ingredients,user_dislike_ingredients,user_spicy_level,user_cooking_level,user_owned_ingredients,user_basic_seasoning,must_use_ingredients
count,10,10,10,10,10,10,10
unique,9,6,3,3,10,10,4
top,['달걀'],['양파'],3단계,초급,"['김치', '계란']","['소금', '후추']",['김치']
freq,2,3,4,4,1,1,4


In [18]:
import pandas as pd

# Recipe와 User 정보를 균등하게 결합하여 JSON 형식으로 변환
def create_combined_dataset(recipeinfo_df, userinfo_df):
    combined_data = []
    len_userinfo = len(userinfo_df)  # UserInfo 데이터 수
    
    for idx, recipe_row in recipeinfo_df.iterrows():
        user_idx = idx % len_userinfo  # user_info를 균등하게 분배하기 위한 인덱스
        user_row = userinfo_df.iloc[user_idx]
        
        combined_entry = {
            "user_info": {
                "user_allergy_ingredients": eval(user_row["user_allergy_ingredients"]),
                "user_dislike_ingredients": eval(user_row["user_dislike_ingredients"]),
                "user_spicy_level": user_row["user_spicy_level"],
                "user_cooking_level": user_row["user_cooking_level"],
                "user_owned_ingredients": eval(user_row["user_owned_ingredients"]),
                "user_basic_seasoning": eval(user_row["user_basic_seasoning"]),
                "must_use_ingredients": eval(user_row["must_use_ingredients"]),
            },
            "recipe_info": {
                "_id": recipe_row["_id"],
                "title": recipe_row["title"],
                "type_key": recipe_row["type_key"],
                "method_key": recipe_row["method_key"],
                "servings": recipe_row["servings"],
                "cooking_time": recipe_row["cooking_time"] if not pd.isna(recipe_row["cooking_time"]) else "",
                "difficulty": recipe_row["difficulty"] if not pd.isna(recipe_row["difficulty"]) else "",
                "ingredients": eval(recipe_row["ingredients"]),
                "cooking_steps": eval(recipe_row["cooking_steps"]),
                "tips": eval(recipe_row["tips"]),
            },
        }
        combined_data.append(combined_entry)
    
    return combined_data

# 데이터셋 생성
combined_dataset = create_combined_dataset(df_recipeinfo, df_userinfo)

# JSON 데이터셋 저장 (옵션)
import json
with open('data/combined_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(combined_dataset, f, ensure_ascii=False, indent=4)

print("데이터셋 생성 완료. 첫 번째 데이터:")
print(combined_dataset[0])
print("갯수:", len(combined_dataset))


데이터셋 생성 완료. 첫 번째 데이터:
{'user_info': {'user_allergy_ingredients': ['땅콩'], 'user_dislike_ingredients': ['양파'], 'user_spicy_level': '3단계', 'user_cooking_level': '초급', 'user_owned_ingredients': ['김치', '계란'], 'user_basic_seasoning': ['소금', '후추'], 'must_use_ingredients': ['김치']}, 'recipe_info': {'_id': '673c36d38fad4e3c931171ba', 'title': '토마토 마리네이드&냉파스타, 새콤달콤 시원한 별미', 'type_key': '샐러드', 'method_key': '절임', 'servings': '1인분', 'cooking_time': '60분 이내', 'difficulty': '초급', 'ingredients': ['방울토마토(1.5kg)', '양파(1개)', '파슬리가루(1스푼)', '올리브오일(10스푼)', '발사믹식초(5스푼)', '레몬즙(2스푼)', '꿀(1.5스푼)', '소금(약간)', '통후추(약간)', '토마토 마리네이드(1컵)', '파스타면(1컵)', '소금(1스푼)', '파마산치즈(1스푼)'], 'cooking_steps': ['양파는 잘게 다져 준비해주세요양파(소) 1개', '방울토마토는 끓는물에 약 2분간 데쳐주세요. 칼집을 내면 좀더 편하긴 한데 안해도 상관없어요방울토마토 1팩(약 1.5키로)중간불', '데친 후에는 물을 버리고 찬물에 담궈 식혀주고 토마토 껍질을 벗겨줍니다.칼집을 내지않아도 이렇게 껍질이 터져서 벗기기 쉽고 터져있지 않더라도 껍질 분리가 잘 된답니다^^', '껍질을 벗긴 토마토와 다진 양파, 파슬리가루를 넣어주세요step1의 다진양파, 파슬리가루 약 1스푼', '올리브오일, 레몬즙, 발사믹식초, 소금, 후추, 꿀을 요리재료에 적힌 분량대로 넣어준 뒤 잘 섞어주세요올리브오일 10스

In [20]:
import pandas as pd

# JSON 파일 경로
file_path = 'data/combined_dataset.json'

# JSON 파일을 DataFrame으로 불러오기
df = pd.read_json(file_path)

df


,user_info,recipe_info
0,"{'user_allergy_ingredients': ['땅콩'], 'user_dis...","{'_id': '673c36d38fad4e3c931171ba', 'title': '..."
1,"{'user_allergy_ingredients': ['새우'], 'user_dis...","{'_id': '673c36d38fad4e3c93117d03', 'title': '..."
2,"{'user_allergy_ingredients': ['달걀'], 'user_dis...","{'_id': '673c36d38fad4e3c93115d44', 'title': '..."
3,"{'user_allergy_ingredients': ['우유'], 'user_dis...","{'_id': '673c36d38fad4e3c93117b22', 'title': '..."
4,"{'user_allergy_ingredients': ['밀가루'], 'user_di...","{'_id': '673c36d38fad4e3c931162f7', 'title': '..."
5,"{'user_allergy_ingredients': ['갑각류'], 'user_di...","{'_id': '673c36d38fad4e3c931181c7', 'title': '..."
6,"{'user_allergy_ingredients': ['고등어'], 'user_di...","{'_id': '673c36d38fad4e3c931171d3', 'title': '..."
7,"{'user_allergy_ingredients': ['호두'], 'user_dis...","{'_id': '673c36d38fad4e3c9311536d', 'title': '..."
8,"{'user_allergy_ingredients': ['달걀'], 'user_dis...","{'_id': '673c36d38fad4e3c93116e67', 'title': '..."
9,"{'user_allergy_ingredients': ['참깨'], 'user_dis...","{'_id': '673c36d28fad4e3c93114bca', 'title': '..."


## 작은 datatset 생성

In [2]:
df[:2]

,user_info,recipe_info
0,"{'user_allergy_ingredients': ['땅콩'], 'user_dis...","{'_id': '673c36d28fad4e3c931146b6', 'title': '..."
1,"{'user_allergy_ingredients': ['땅콩'], 'user_dis...","{'_id': '673c36d28fad4e3c931146b7', 'title': '..."


In [ ]:
import json
with open('combined_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(combined_dataset, f, ensure_ascii=False, indent=4)